In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd

In [2]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
df = pd.read_csv('./dados/cleaned_dataset.csv')

In [4]:
df = df.drop(columns=['Id'])

In [5]:
X = df[['Score', 'Gender', 'Age', 'Assets', 'Products', 'Active']]
y = df['Churned']

In [6]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1).to(device)  # Reshape y for compatibility

In [7]:
X_tensor.shape

torch.Size([999, 6])

In [8]:
y_tensor.shape

torch.Size([999, 1])

In [9]:
# Split data into training and testing sets
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = train_test_split(X_tensor, y_tensor, test_size=0.1, random_state=42)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
# Define neural network model
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, 1)
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.gelu(self.fc1(x))
        x = self.gelu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [11]:
# Function to train and evaluate the model using cross-validation

def cross_val_model(hidden1_size, hidden2_size, learning_rate):
    hidden1_size = int(hidden1_size)
    hidden2_size = int(hidden2_size)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_accuracies = []

    for train_index, val_index in kf.split(X_tensor):
        X_train_fold = X_tensor[train_index]
        y_train_fold = y_tensor[train_index]
        X_val_fold = X_tensor[val_index]
        y_val_fold = y_tensor[val_index]

        # create DataLoader for training fold
        train_dataset = TensorDataset(X_train_fold, y_train_fold)
        train_loader = DataLoader(train_dataset, batch_size=9, shuffle=True)

        # Initialize the model, criterion, and optmizer
        model = BinaryClassificationModel(X_train_fold.shape[1], hidden1_size, hidden2_size).to(device)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Train the model
        model.train()
        for epoch in range(50):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Evaluate the model on the validation fold
        model.eval()
        with torch.no_grad():
            y_pred_probs = model(X_val_fold)
            y_pred = (y_pred_probs > 0.5).float()

        # Calculate accuracy for this fold
        acc = accuracy_score(y_val_fold.cpu().numpy(), y_pred.cpu().numpy())
        cv_accuracies.append(acc)

    # Return the average accuracy across folds
    return np.mean(cv_accuracies)

In [12]:
# Set the parameter bounds for Bayesian Optimization
param_bounds = {
    'hidden1_size': (32, 128),  # Number of neurons in the first hidden layer
    'hidden2_size': (16, 64),   # Number of neurons in the second hidden layer
    'learning_rate': (0.0001, 0.01)  # Learning rate for training
}


In [13]:
# Run Bayesian Optimization
optimizer = BayesianOptimization(f=cross_val_model, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

|   iter    |  target   | hidden... | hidden... | learni... |
-------------------------------------------------------------
| 1         | 0.4412    | 67.96     | 61.63     | 0.007347  |
| 2         | 0.7958    | 89.47     | 23.49     | 0.001644  |
| 3         | 0.6798    | 37.58     | 57.58     | 0.006051  |
| 4         | 0.5692    | 99.97     | 16.99     | 0.009702  |
| 5         | 0.7968    | 111.9     | 26.19     | 0.0019    |
| 6         | 0.5492    | 111.8     | 26.15     | 0.006438  |
| 7         | 0.7838    | 47.44     | 35.15     | 0.0009565 |
| 8         | 0.6768    | 63.22     | 39.0      | 0.001514  |
| 9         | 0.5552    | 122.5     | 33.66     | 0.002291  |
| 10        | 0.3252    | 117.8     | 57.35     | 0.004191  |
| 11        | 0.6948    | 63.9      | 48.64     | 0.009634  |
| 12        | 0.5552    | 41.07     | 54.46     | 0.005997  |
| 13        | 0.5432    | 70.69     | 56.35     | 0.0076    |
| 14        | 0.4448    | 90.59     | 26.63     | 0.009764  |
| 15    

In [14]:
# Print the best parameters found
print("Best Parameters Found:")
print(optimizer.max)

Best Parameters Found:
{'target': 0.7968140703517588, 'params': {'hidden1_size': 111.91449351684048, 'hidden2_size': 26.192277312557255, 'learning_rate': 0.0019000671753502962}}


In [15]:
# Train the final model with the best parameters found and evaluate it
best_params = optimizer.max['params']
model = BinaryClassificationModel(X_tensor.shape[1], int(best_params['hidden1_size']), int(best_params['hidden2_size'])).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])
criterion = nn.BCELoss()

In [16]:
# Create a DataLoader for training the final model
train_dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [17]:
# Train the model using the full dataset
model.train()
for epoch in range(50):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [18]:
# Evaluate the final model on the test set
model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_tensor.to(device))
    y_pred = (y_pred_probs > 0.5).float()

In [20]:
# Calculate metrics for the final model
accuracy = accuracy_score(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
print("\nTest Accuracy:", accuracy)


Test Accuracy: 0.77


In [21]:
# Compute and format the confusion matrix
cnf_matrix = confusion_matrix(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy())
cnf_table = pd.DataFrame(data=cnf_matrix, index=['Non-Churned', 'Churned'], columns=['Non_churned(pred)', 'Churned(pred)'])

print("\nConfusion Matrix:")
print(cnf_table)


Confusion Matrix:
             Non_churned(pred)  Churned(pred)
Non-Churned                 75              4
Churned                     19              2


In [22]:
# Print the classification report with target names
print("\nClassification Report:")
print(classification_report(y_test_tensor.cpu().numpy(), y_pred.cpu().numpy(), target_names=['Non Churned', 'Churned']))


Classification Report:
              precision    recall  f1-score   support

 Non Churned       0.80      0.95      0.87        79
     Churned       0.33      0.10      0.15        21

    accuracy                           0.77       100
   macro avg       0.57      0.52      0.51       100
weighted avg       0.70      0.77      0.72       100

